In [1]:
#| default_exp 38_oak-distilbert-for-msmarco-from-scratch

In [2]:
%reload_ext autoreload
%autoreload 2

In [4]:
#| export
import os,torch,json, torch.multiprocessing as mp, joblib, numpy as np, scipy.sparse as sp

from transformers import DistilBertConfig

from xcai.basics import *
from xcai.models.oak import OAK016

In [3]:
os.environ['WANDB_MODE'] = 'disabled'

In [4]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT'] = 'mogicX_00-msmarco'

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/38_oak-distilbert-for-msmarco-from-scratch'
    meta_embed_init_file = None
    
    input_args = parse_args()

    if input_args.exact:
        config_file = '/home/scai/phd/aiz218323/scratch/datasets/msmarco/XC/configs/oak-for-msmarco-with-hard-negatives.json'
        config_key = 'data'
    else:
        raise NotImplementedError('Create a configuration file for using all the labels.')

    mname, meta_name = 'distilbert-base-uncased', 'lnk'
    meta_embed_init_file = None

    pkl_file = get_pkl_file(input_args.pickle_dir, 'msmarco_data-oak-for-msmarco-with-hard-negatives_distilbert-base-uncased', input_args.use_sxc_sampler, 
                            input_args.exact, input_args.only_test)

    do_inference = input_args.do_train_inference or input_args.do_test_inference or input_args.save_train_prediction or input_args.save_test_prediction or input_args.save_representation

    os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
    block = build_block(pkl_file, config_file, input_args.use_sxc_sampler, config_key, do_build=input_args.build_block, 
                        only_test=input_args.only_test, main_oversample=True, meta_oversample={'lnk_meta':False, 'neg_meta':True}, 
                        n_slbl_samples=1, n_sdata_meta_samples={'lnk_meta':5, 'neg_meta':1}, return_scores=True, 
                        train_meta_topk={"lnk_meta":5}, test_meta_topk={"lnk_meta":5})

    args = XCLearningArguments(
        output_dir=output_dir,
        logging_first_step=True,
        per_device_train_batch_size=800,
        per_device_eval_batch_size=800,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        save_strategy="steps",
        eval_strategy="steps",
        eval_steps=500,
        save_steps=500,
        save_total_limit=5,
        num_train_epochs=30,
        predict_with_representation=True,
        adam_epsilon=1e-6,
        warmup_steps=100,
        weight_decay=0.01,
        learning_rate=6e-5,
        representation_search_type='BRUTEFORCE',
    
        output_representation_attribute='data_fused_repr',
        label_representation_attribute='data_repr',
        metadata_representation_attribute='data_repr',
        data_augmentation_attribute='data_repr',
        representation_attribute='data_fused_repr',
        clustering_representation_attribute='data_fused_repr',

        group_by_cluster=True,
        num_clustering_warmup_epochs=10,
        num_cluster_update_epochs=5,
        num_cluster_size_update_epochs=25,
        use_data_metadata_for_clustering=True,
        clustering_type='EXPO',
        minimum_cluster_size=2,
        maximum_cluster_size=1600,

        metric_for_best_model='P@1',
        load_best_model_at_end=True,
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
    
        use_distributional_representation=False,
        use_encoder_parallel=True,
        max_grad_norm=None,
        fp16=True,
        
        label_names=[f'{meta_name}2data_idx', f'{meta_name}2data_data2ptr'],

        prune_metadata=False,
        num_metadata_prune_warmup_epochs=10,
        num_metadata_prune_epochs=5,
        metadata_prune_batch_size=2048,
        prune_metadata_names=[f'{meta_name}_meta'],
        use_data_metadata_for_pruning=True,
    
        predict_with_augmentation=False,
        use_augmentation_index_representation=True,
    
        data_aug_meta_name=meta_name,
        augmentation_num_beams=None,
        data_aug_prefix=meta_name,
        use_label_metadata=False,

        data_meta_batch_size=2048,
        augment_metadata=False,
        num_metadata_augment_warmup_epochs=10,
        num_metadata_augment_epochs=5,
    
        use_cpu_for_searching=True,
        use_cpu_for_clustering=True,
    )
        
    def model_fn(mname):
        model = OAK016.from_pretrained(mname, margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                                       data_aug_meta_prefix=f'{meta_name}2data', lbl2data_aug_meta_prefix=None,
                                       neg2data_aug_meta_prefix=None,
                                       
                                       num_metadata=block.train.dset.meta[f'{meta_name}_meta'].n_meta, resize_length=5000,
                                       
                                       calib_margin=0.05, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                                       calib_loss_weight=0.1, use_calib_loss=True,
        
                                       use_query_loss=True,
                                       
                                       use_encoder_parallel=True, normalize=False)
        return model
    
    def init_fn(model):
        model.init_retrieval_head()
        # model.init_cross_head()
        model.init_meta_embeddings()

        meta_embeddings = torch.tensor(np.load(meta_embed_init_file), dtype=torch.float32)
        model.encoder.set_pretrained_meta_embeddings(meta_embeddings)
        model.encoder.freeze_pretrained_meta_embeddings()

    metric = PrecReclMrr(block.test.dset.n_lbl, block.test.data_lbl_filterer, pk=10, rk=200, rep_pk=[1, 3, 5, 10], 
                         rep_rk=[10, 100, 200], mk=[5, 10, 20])

    bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

    model = load_model(args.output_dir, model_fn, {"mname": mname}, init_fn, do_inference=do_inference, 
                       use_pretrained=input_args.use_pretrained)
    
    learn = XCLearner(
        model=model,
        args=args,
        train_dataset=block.train.dset,
        eval_dataset=block.test.dset,
        data_collator=block.collator,
        compute_metrics=metric,
    )
    
    main(learn, input_args, n_lbl=block.test.dset.n_lbl)
    